In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ..

/home/suriya/dcu/Group-Project


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [7]:
hyperparams = {'Friis': {'Gr_dBi': -3.464561870341017,
  'Gt_dBi': 0.20597726357898227,
  'L': 2.1516474937337478,
  'Pt_dBm': 17.87357752441881,
  'f': 2403652319.6880946,
  'n': 2.930021915116384},
 'LinearApprox': {'N': 2.915682422144732, 'TX': 11.682119667187617},
 'LogNormal': {'Gr_dBi': -5.125678676280162,
  'Gt_dBi': -4.108101510558816,
  'L': 0.7656207801852686,
  'Pt_dBm': -1.8532796868212498,
  'f': 2463516474.7305307,
  'n': 3.7519240746066163}}

In [8]:
from src.featutils import aggregate_features_from_folder
from src.features import radioprop

train_dir = "data/tc4tl_training_data_v1/tc4tl/data/train/"
train_key = pd.read_csv("data/tc4tl_training_data_v1/tc4tl/docs/tc4tl_train_key.tsv", sep="\t")
test_dir = "data/tc4tl_data_v5/tc4tl/data/test/"
test_key = pd.read_csv("data/tc4tl_test_key/tc4tl/docs/tc4tl_test_key.tsv", sep="\t")
dev_dir = "data/tc4tl_data_v5/tc4tl/data/dev/"
dev_key = pd.read_csv("data/tc4tl_data_v5/tc4tl/docs/tc4tl_dev_key.tsv", sep="\t")

trainset, pipe = aggregate_features_from_folder(train_dir, train_key, feat_fn=radioprop.extract_features,
                               postproc_fn=radioprop.postproc_feature_dicts, verbose=True, testing=0,
                               tunables=hyperparams)
feat_cols = [col for col in trainset.columns if col != 'DistanceFloat' ]
tr_X, tr_y = trainset[feat_cols], trainset['DistanceFloat']

Post-processing: zzzjbxhg_tc4tl20.csv: 100%|██████████| 15552/15552 [00:20<00:00, 747.19it/s]


In [24]:
devset, _ = aggregate_features_from_folder(dev_dir, dev_key, feat_fn=radioprop.extract_features, pipe=pipe,
                               postproc_fn=radioprop.postproc_feature_dicts, verbose=True, testing=False,
                               tunables=hyperparams)

Post-processing: zzxttrqc_tc4tl20.csv: 100%|██████████| 935/935 [00:01<00:00, 786.25it/s]


In [26]:
testset, _ = aggregate_features_from_folder(test_dir, test_key, feat_fn=radioprop.extract_features, pipe=pipe,
                               postproc_fn=radioprop.postproc_feature_dicts, verbose=True, testing=False,
                               tunables=hyperparams)

Post-processing: zzzgwsci_tc4tl20.csv: 100%|██████████| 8423/8423 [00:11<00:00, 747.78it/s]


## Cluster training set

In [57]:
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

pca = PCA(10)
X_pca = pca.fit_transform(tr_X)
kmeans = KMeans(n_clusters=20, random_state=69)
label = kmeans.fit_predict(X_pca)
dev_pca = pca.transform(devset[feat_cols])
dlabels = kmeans.predict(dev_pca)
test_pca = pca.transform(testset[feat_cols])
tlabels = kmeans.predict(test_pca)
cluster_labels = np.unique(label)
test_mse, dev_mse = [], []
for i in cluster_labels:
    X, y = X_pca[label == i], tr_y.values[label==i]
    # test_X = test_pca[feat_cols].values[tlabels == i]
    test_X, test_y = test_pca[tlabels == i], testset["DistanceFloat"][tlabels==i]
    dev_X, dev_y = dev_pca[dlabels == i], devset["DistanceFloat"][dlabels==i]
    rf = RandomForestRegressor()
    rf.fit(X, y)
    if test_X.shape[0] > 0:
        test_mse.append(mean_squared_error(test_y, rf.predict(test_X)))
        print(i, "MSE <TEST>: ", test_mse[-1], "|", test_X.shape[0])
    if dev_X.shape[0] > 0:
        dev_mse.append(mean_squared_error(dev_y, rf.predict(dev_X)))
        print(" MSE <DEV>: ", dev_mse[-1], "|", dev_X.shape[0])

0 MSE <TEST>:  2.6259902869249396 | 826
 MSE <DEV>:  2.0950094166666666 | 108
1 MSE <TEST>:  0.3900049678899101 | 218
 MSE <DEV>:  0.4895810526315816 | 19
2 MSE <TEST>:  1.818417599999995 | 60
 MSE <DEV>:  0.8608827272727259 | 11
3 MSE <TEST>:  0.1309902631578952 | 76
 MSE <DEV>:  0.21358800000000058 | 2
4 MSE <TEST>:  1.625291891019174 | 991
 MSE <DEV>:  1.7511180352941187 | 85
5 MSE <TEST>:  3.328084443396227 | 106
 MSE <DEV>:  3.635384142857142 | 21
6 MSE <TEST>:  3.310854885 | 200
 MSE <DEV>:  3.1534946666666674 | 27
7 MSE <TEST>:  1.662523790175171 | 2626
 MSE <DEV>:  1.6232951739130428 | 276
8 MSE <TEST>:  0.3597642373540856 | 771
 MSE <DEV>:  0.38362340186915844 | 107
9 MSE <TEST>:  2.242394013698626 | 73
 MSE <DEV>:  0.6966324000000016 | 5
10 MSE <TEST>:  1.8925251670588237 | 425
 MSE <DEV>:  2.026513963636364 | 55
12 MSE <TEST>:  0.018388800000000163 | 130
 MSE <DEV>:  0.007855200000000126 | 15
13 MSE <TEST>:  2.8398992587956425e-29 | 74
 MSE <DEV>:  2.8398992587956425e-29 | 6

In [56]:
np.array(test_mse).mean(), np.array(dev_mse).mean()

(1.411763732589411, 1.5986182728244165)

1.7512280225068646

In [19]:
tr_X

,LinearApprox:Hist_0.15,LinearApprox:Hist_0.25,LinearApprox:Hist_0.35,LinearApprox:Hist_0.45,LinearApprox:Hist_0.55,LinearApprox:Hist_0.65,LinearApprox:Hist_0.75,LinearApprox:Hist_0.85,LinearApprox:Hist_0.95,LinearApprox:Hist_1.05,...,LogNormal:Hist_4.15,LogNormal:Hist_4.25,LogNormal:Hist_4.35,LogNormal:Hist_4.45,LogNormal:Hist_4.55,LogNormal:Hist_4.65,LogNormal:Hist_4.75,LogNormal:Hist_4.85,LogNormal:Hist_4.95,CoarseGrain
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.156171,0.0,0.204698,0.0,0.0,0.027650,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.004608,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.088161,0.0,0.174497,0.0,0.0,0.052995,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.151134,0.0,0.107383,0.0,0.0,0.050691,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15547,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.022670,0.0,0.013423,0.0,0.0,0.016129,0.0,1.0
15548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.068010,0.0,0.137584,0.0,0.0,0.085253,0.0,1.0
15549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.221662,0.0,0.238255,0.0,0.0,0.211982,0.0,1.0
15550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.032746,0.0,0.053691,0.0,0.0,0.046083,0.0,0.0
